In [1]:
!wget https://storage.googleapis.com/ibeans/train.zip

--2021-02-11 12:17:08--  https://storage.googleapis.com/ibeans/train.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.122.128, 172.253.63.128, 142.250.31.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.122.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 143812152 (137M) [application/zip]
Saving to: ‘train.zip’

train.zip           100%[===================>] 137.15M   116MB/s    in 1.2s    

2021-02-11 12:17:09 (116 MB/s) - ‘train.zip’ saved [143812152/143812152]



In [2]:
! unzip train.zip -d train
!ls train/

Archive:  train.zip
   creating: train/train/
   creating: train/train/healthy/
   creating: train/train/angular_leaf_spot/
   creating: train/train/bean_rust/
  inflating: train/train/healthy/healthy_train.265.jpg  
  inflating: train/train/healthy/healthy_train.271.jpg  
  inflating: train/train/healthy/healthy_train.259.jpg  
  inflating: train/train/healthy/healthy_train.99.jpg  
  inflating: train/train/healthy/healthy_train.311.jpg  
  inflating: train/train/healthy/healthy_train.66.jpg  
  inflating: train/train/healthy/healthy_train.72.jpg  
  inflating: train/train/healthy/healthy_train.305.jpg  
  inflating: train/train/healthy/healthy_train.339.jpg  
  inflating: train/train/healthy/healthy_train.113.jpg  
  inflating: train/train/healthy/healthy_train.107.jpg  
  inflating: train/train/healthy/healthy_train.106.jpg  
  inflating: train/train/healthy/healthy_train.112.jpg  
  inflating: train/train/healthy/healthy_train.338.jpg  
  inflating: train/train/healthy/healthy_trai

In [3]:
!nvidia-smi


Thu Feb 11 12:17:19 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!pip install tensorflow-gpu

     |████████████████████████████████| 394.3MB 44kB/s 


In [5]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

In [6]:
!wget https://storage.googleapis.com/ibeans/test.zip

--2021-02-11 12:18:23--  https://storage.googleapis.com/ibeans/test.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.8.16, 172.217.13.80, 172.217.13.240, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.8.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17708541 (17M) [application/zip]
Saving to: ‘test.zip’

test.zip            100%[===================>]  16.89M  --.-KB/s    in 0.09s   

2021-02-11 12:18:23 (191 MB/s) - ‘test.zip’ saved [17708541/17708541]



In [7]:
! unzip test.zip -d test
!ls test/

Archive:  test.zip
   creating: test/test/
   creating: test/test/healthy/
   creating: test/test/angular_leaf_spot/
   creating: test/test/bean_rust/
  inflating: test/test/healthy/healthy_test.21.jpg  
  inflating: test/test/healthy/healthy_test.35.jpg  
  inflating: test/test/healthy/healthy_test.34.jpg  
  inflating: test/test/healthy/healthy_test.20.jpg  
  inflating: test/test/healthy/healthy_test.36.jpg  
  inflating: test/test/healthy/healthy_test.22.jpg  
  inflating: test/test/healthy/healthy_test.23.jpg  
  inflating: test/test/healthy/healthy_test.37.jpg  
  inflating: test/test/healthy/healthy_test.8.jpg  
  inflating: test/test/healthy/healthy_test.33.jpg  
  inflating: test/test/healthy/healthy_test.27.jpg  
  inflating: test/test/healthy/healthy_test.26.jpg  
  inflating: test/test/healthy/healthy_test.32.jpg  
  inflating: test/test/healthy/healthy_test.9.jpg  
  inflating: test/test/healthy/healthy_test.24.jpg  
  inflating: test/test/healthy/healthy_test.30.jpg  
  i

In [8]:

# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/train/train'
valid_path = '/test/test'

In [9]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

87916544/87910968 [==============================] - 0s 0us/step


In [10]:

# don't train existing weights
for layer in inception.layers:
    layer.trainable = False

In [11]:
# useful for getting number of output classes
folders = glob('train/train/*')

In [12]:
folders

['train/train/healthy',
 'train/train/bean_rust',
 'train/train/angular_leaf_spot']

In [13]:
# our layers - you can add more if you want
x = Flatten()(inception.output)


In [14]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=inception.input, outputs=prediction)

In [15]:
# view the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [16]:

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [17]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [18]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('train/train/',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1034 images belonging to 3 classes.


In [20]:
labels = (training_set.class_indices)
print(labels)

{'angular_leaf_spot': 0, 'bean_rust': 1, 'healthy': 2}


In [21]:
test_set = test_datagen.flow_from_directory('test/test/',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')


Found 128 images belonging to 3 classes.


In [22]:
labels = (test_set.class_indices)
print(labels)


{'angular_leaf_spot': 0, 'bean_rust': 1, 'healthy': 2}


In [23]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
33/33 [==============================] - 114s 3s/step - loss: 9.9020 - accuracy: 0.4565 - val_loss: 2.1591 - val_accuracy: 0.6797
Epoch 2/10
33/33 [==============================] - 109s 3s/step - loss: 1.9230 - accuracy: 0.7620 - val_loss: 0.9932 - val_accuracy: 0.8281
Epoch 3/10
33/33 [==============================] - 109s 3s/step - loss: 0.8477 - accuracy: 0.8482 - val_loss: 1.1452 - val_accuracy: 0.7969
Epoch 4/10
33/33 [==============================] - 108s 3s/step - loss: 0.6952 - accuracy: 0.8797 - val_loss: 1.1026 - val_accuracy: 0.7578
Epoch 5/10
33/33 [==============================] - 107s 3s/step - loss: 0.5968 - accuracy: 0.8727 - val_loss: 1.0946 - val_accuracy: 0.8047
Epoch 6/10
33/33 [==============================] - 107s 3s/step - loss: 0.4685 - accuracy: 0.8863 - val_loss: 0.8537 - val_accuracy: 0.8359
Epoch 7/10
33/33 [==============================] - 106s 3s/step - loss: 0.3998 - accuracy: 0.9108 - val_loss: 2.1502 - val_accuracy: 0.6719
Epoch 8/10
33

In [24]:
from keras.models import load_model

model.save('/content/drive/MyDrive/Final models/beans/beansINC.h5')  
/content/test/test/

In [34]:
import os
from google.colab.patches import cv2_imshow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
disease =['angular_leaf_spot', 'bean_rust', 'healthy']
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt

import matplotlib.image as mpimg
#img = mpimg.imread('your_image.png')
dir_path="/content/test/test/"
k=0
count=0
for y in range(0, 2):
  print(disease[y])
  for i in os.listdir(dir_path+disease[y]):
    k=k+1
    img = image.load_img(dir_path+disease[y]+"/"+i, target_size=(224, 224))
    imgplot = plt.imshow(img)
    plt.show()
    # Preprocessing the image
    x = image.img_to_array(img)
    # x = np.true_divide(x, 255)
    ## Scaling
    x=x/255
    x = np.expand_dims(x, axis=0)
    preds = model.predict(x)
    if disease[np.argmax(preds)] ==str(disease[y]):
      count=count+1
    print(disease[np.argmax(preds)])
  print("total images :"+str(k))
  print("Correctly predicted :"+str(count))
  print(str(float(count/k)*float(100))+ "%") 
  print("")

Output hidden; open in https://colab.research.google.com to view.